# Box Hubs Q&A with Box AI

In this notebook, we will learn how to use the Box AI API `/ask` endpoint with a Box Hub to ask questions across a curated list of files in Box. Specifically, we will have access to a a series of documents related to a clinical drug trial. 

## Prerequisites

You must have completed the Setup notebook first. This will create all of the Box objects, folders, and files that you need, and will have created an environment file to help you get started and import all the libraries you will need.

## Workshop

The first step is to import all of the environment variables we need for this exercise.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_HUBS_ID=os.getenv('HUBS_ID')

print("BOX_HUBS_ID:", BOX_HUBS_ID)

Next we will grab the BoxClient object from the Python SDK to authenticate ourselves to the API. We'll print out the current user's information to ensure we are properly authenticated.

In [ ]:
from box_sdk_gen import BoxClient, CCGConfig, BoxCCGAuth

ccg_config = CCGConfig(
    client_id=BOX_CLIENT_ID,
    client_secret=BOX_CLIENT_SECRET,
    user_id=BOX_USER_ID,
)

ccg_auth = BoxCCGAuth(ccg_config)

client = BoxClient(ccg_auth)

print(f"{client.users.get_user_me()}")

Now we will use the Box Python SDK to call the `/ask` endpoint to analyze the drug trial and create a summary of the approach used to test the drug, Novostatin.

In [ ]:
from box_sdk_gen import (
    CreateAiAskMode,
    AiItemAsk,
    AiItemAskTypeField,
    AiDialogueHistory
)

prompt = """
Analyze the drug trial and create a summary of the approach used to test the drug.
"""

ai_response = client.ai.create_ai_ask(
    mode=CreateAiAskMode.SINGLE_ITEM_QA,
    prompt=prompt,
    items=[
        AiItemAsk(
            id=BOX_HUBS_ID, 
            type=AiItemAskTypeField.HUBS,
        )
    ]
)

print (ai_response.answer)

We now have a pretty good summary of the methodology used to test the drug, but we might have more questions. Let's implement dialogue_history to keep the conversation going without losing context from previous questions and answers.

In [ ]:
dialogue_history = []

dialogue_history.append(AiDialogueHistory(
    prompt=prompt,
    answer=ai_response.answer, # type: ignore
    created_at=ai_response.created_at # type: ignore
))

new_response = client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    "Did the drug work better than a placebo?",
    [
        AiItemAsk(
            id=BOX_HUBS_ID,
            type=AiItemAskTypeField.HUBS,
        )
    ],
    dialogue_history=dialogue_history,
    include_citations=True
)

print (new_response.answer)
print (f"Citations: {new_response.citations}")

With Box Hubs and Box AI, you can get powerful insights across large, curated sets of data, without having to ingest, index, and sync files, without paying large infrastructure costs, and without having to maintain it all. Let us do the hard part.

To get you started, we've provided full, runnable files for this exercise. Running the following cells will generate the file for you in the exercise folders. Use it as-is, or use it as inspiration or a starting point for your workflows and applications.

In [ ]:
%%writefile box_ai_qna_hub.py
import asyncio
import os

from dotenv import load_dotenv

from box_sdk_gen import (
    AiDialogueHistory,
    AiItemAsk,
    AiItemAskTypeField,
    BoxClient,
    BoxCCGAuth,
    CCGConfig,
    CreateAiAskMode
)

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_HUBS_ID=os.getenv('HUBS_ID')

async def chat_with_ai(): 
    ccg_config = CCGConfig(
        client_id=BOX_CLIENT_ID,
        client_secret=BOX_CLIENT_SECRET,
        user_id=BOX_USER_ID,
    )
    auth = BoxCCGAuth(config=ccg_config)
    client = BoxClient(auth=auth)
    
    dialogue_history = []
    
    print("Box AI Chat - Type 'quit' to exit")
    print("-" * 40)
    
    while True:
        prompt = input("\nYou: ").strip()
        
        if prompt.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
            
        if not prompt:
            continue
            
        try:
            response = client.ai.create_ai_ask(
                mode=CreateAiAskMode.SINGLE_ITEM_QA,
                prompt=prompt,
                items=[
                    AiItemAsk(
                        id=BOX_HUBS_ID, 
                        type=AiItemAskTypeField.HUBS,
                    )
                ],
                dialogue_history=dialogue_history,
                include_citations=True,
            )
            
            print(f"\nAI: {response.answer}") # type: ignore
            
            i=1
            print("\nCitations:")
            print("-" * 20)
            for citation in response.citations:
                print(f"\n{i}. {citation.content} ({citation.name})") # type: ignore
                i += 1
            
            # Update dialogue history
            dialogue_history.append(AiDialogueHistory(
                prompt=prompt,
                answer=response.answer, # type: ignore
                created_at=response.created_at # type: ignore
            ))
            
        except Exception as e:
            print(f"\nError: {e}")

if __name__ == "__main__":
    asyncio.run(chat_with_ai())
